In [1]:
import numpy as np
from sklearn import tree
from sklearn import model_selection
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
from keras.callbacks import ModelCheckpoint
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import GridSearchCV
import pandas as pd
import math

%matplotlib inline

Using TensorFlow backend.


In [17]:
dataset =pd.read_csv('ddgokber.csv')

In [18]:
# Correlation Matrix Heatmap
f, ax = plt.subplots(figsize=(10, 6))
corr = dataset.corr()
hm = sns.heatmap(round(corr,2), annot=True, ax=ax, cmap="coolwarm",fmt='.2f',
                 linewidths=.05)
f.subplots_adjust(top=0.93)
t= f.suptitle('Features Correlation Heatmap', fontsize=14)

In [19]:
# Get one hot encoding
one_hot = pd.get_dummies(dataset['food_label'])
# Drop columns as it is now encoded
dataset = dataset.drop('food_label',axis = 1)
# Join the encoded df
dataset = dataset.join(one_hot)
dataset

image_name  Side_coinWidth  Side_coinHeigth  Top_coinWidth  \
0        orange              79               78             66   
1        orange              63               58             63   
2        orange              61               58             65   
3        orange              75               72             71   
4        orange              61               63             60   
5        orange              71               69             63   
6        orange              75               75             63   
7        orange              62               59             71   
8        orange              61               60             64   
9        orange              60               60             71   
10       orange              62               59             65   
11       orange              64               60             58   
12       orange              70               67             70   
13       orange              59               59             66   
14       orange              64               60             70   
15       orange              61               58             64   
16       orange              60               58             68   
17       orange              63               60             68   
18       orange              61               58             72   
19       orange              58               58             67   
20       orange              60               59             71   
21       orange              60               57             61   
22       orange              62               58             64   
23       orange              60               60             61   
24       orange              60               58             63   
25       orange              60               60             63   
26       orange              61               58             60   
27       orange              61               59             65   
28       orange              60               59             63   
29       orange              60               58             61   
...         ...             ...              ...            ...   
1378    sachima              77               72             57   
1379    sachima              78               75             71   
1380      peach              63               59             75   
1381      peach              73               69             65   
1382      peach              66               65             62   
1383      peach              63               58             69   
1384      lemon              62               63             73   
1385      lemon              62               61             66   
1386      lemon              89               87             73   
1387      lemon              60               60             78   
1388      lemon              60               56             76   
1389     tomato              64               60             74   
1390     tomato              61               60             67   
1391     tomato              62               61             64   
1392     tomato              65               61             72   
1393     tomato              67               63             68   
1394     tomato              69               65             65   
1395       plum              76               72             71   
1396       plum              80               74             80   
1397       plum              63               64             64   
1398       plum              65               64             69   
1399       plum              71               68             70   
1400       plum              69               67             73   
1401       plum              73               70             78   
1402       pear              64               64             61   
1403       pear              72               68             58   
1404       pear              47               49             33   
1405       pear              63               65    

In [20]:
a = dataset.groupby(['image_name'])
b =  a.mean()['realDensity'].to_frame()
dataset = dataset.drop('realDensity', axis=1)
dataset = dataset.merge(b, on='image_name', how='left')

In [21]:
dataset

image_name  Side_coinWidth  Side_coinHeigth  Top_coinWidth  \
0        orange              79               78             66   
1        orange              63               58             63   
2        orange              61               58             65   
3        orange              75               72             71   
4        orange              61               63             60   
5        orange              71               69             63   
6        orange              75               75             63   
7        orange              62               59             71   
8        orange              61               60             64   
9        orange              60               60             71   
10       orange              62               59             65   
11       orange              64               60             58   
12       orange              70               67             70   
13       orange              59               59             66   
14       orange              64               60             70   
15       orange              61               58             64   
16       orange              60               58             68   
17       orange              63               60             68   
18       orange              61               58             72   
19       orange              58               58             67   
20       orange              60               59             71   
21       orange              60               57             61   
22       orange              62               58             64   
23       orange              60               60             61   
24       orange              60               58             63   
25       orange              60               60             63   
26       orange              61               58             60   
27       orange              61               59             65   
28       orange              60               59             63   
29       orange              60               58             61   
...         ...             ...              ...            ...   
1378    sachima              77               72             57   
1379    sachima              78               75             71   
1380      peach              63               59             75   
1381      peach              73               69             65   
1382      peach              66               65             62   
1383      peach              63               58             69   
1384      lemon              62               63             73   
1385      lemon              62               61             66   
1386      lemon              89               87             73   
1387      lemon              60               60             78   
1388      lemon              60               56             76   
1389     tomato              64               60             74   
1390     tomato              61               60             67   
1391     tomato              62               61             64   
1392     tomato              65               61             72   
1393     tomato              67               63             68   
1394     tomato              69               65             65   
1395       plum              76               72             71   
1396       plum              80               74             80   
1397       plum              63               64             64   
1398       plum              65               64             69   
1399       plum              71               68             70   
1400       plum              69               67             73   
1401       plum              73               70             78   
1402       pear              64               64             61   
1403       pear              72               68             58   
1404       pear              47               49             33   
1405       pear              63               65    

In [22]:
# Get one hot encoding
one_hot = pd.get_dummies(dataset['image_name'])
# Drop columns as it is now encoded
dataset = dataset.drop('image_name',axis = 1)
# Join the encoded df
dataset = dataset.join(one_hot)
dataset.head()

Side_coinWidth  Side_coinHeigth  Top_coinWidth  Top_coinHeigth  \
0              79               78             66              65   
1              63               58             63              60   
2              61               58             65              64   
3              75               72             71              66   
4              61               63             60              58   

   Side_foodWidth  Side_foodHeigth  Top_foodWidth  Top_foodHeigth  \
0             232              187            202             222   
1             192              181            204             217   
2             175              181            204             224   
3             201              206            222             208   
4             186              166            197             189   

   Side_coinForeground_pixel  Top_coinForeground_pixel   ...    litchi  mango  \
0                       4411                      2915   ...         0      0   
1                       2474                      2840   ...         0      0   
2                       2575                      2695   ...         0      0   
3                       4004                      2900   ...         0      0   
4                       2734                      2150   ...         0      0   

   mooncake  orange  peach  pear  plum  qiwi  sachima  tomato  
0         0       1      0     0     0     0        0       0  
1         0       1      0     0     0     0        0       0  
2         0       1      0     0     0     0        0       0  
3         0       1      0     0     0     0        0       0  
4         0       1      0     0     0     0        0       0  

[5 rows x 37 columns]

In [23]:
# Drop the faulty grabcut outputs
dataset = dataset.drop(dataset[dataset['Top_coinForeground_pixel'] < 1000].index)
dataset = dataset.drop(dataset[dataset['Side_coinForeground_pixel'] < 1000].index)
dataset = dataset.drop(dataset[dataset['Top_foodForeground_pixel'] < 1000].index)
dataset = dataset.drop(dataset[dataset['Side_foodForeground_pixel'] < 1000].index)

In [32]:
# Combine and create new features

dataset["top_ratio"] = (dataset['Top_foodHeigth'] * dataset['Top_foodWidth']) / (dataset['Top_coinHeigth'] * dataset['Top_coinWidth'])
dataset['side_ratio'] = (dataset['Side_foodWidth'] * dataset['Side_foodHeigth']) / (dataset['Side_coinHeigth'] * dataset['Side_coinWidth'])
dataset['top_ratio_pixels'] = (dataset['Top_foodForeground_pixel'] / dataset['Top_coinForeground_pixel'])
dataset['side_ratio_pixels'] = (dataset['Side_foodForeground_pixel'] / dataset['Side_coinForeground_pixel'])

dataset['top_coin_ratio'] = 2.5 / ((dataset['Top_coinWidth'] + dataset['Top_coinHeigth']) ) 
dataset['side_coin_ratio'] = 2.5 / ((dataset['Side_coinWidth'] + dataset['Side_coinHeigth'])) 

dataset['top_area'] = (dataset['Top_foodHeigth'] * dataset['Top_foodWidth']) * dataset['top_coin_ratio'] * dataset['top_ratio_pixels']
dataset['side_area'] = (dataset['Side_foodHeigth'] * dataset['Side_foodWidth']) * dataset['side_coin_ratio'] * dataset['side_ratio_pixels']

dataset['kcal'] = dataset['real_calorie'] / (dataset['realDensity'] * dataset['realVolume'])

dataset.head()

Side_coinWidth  Side_coinHeigth  Top_coinWidth  Top_coinHeigth  \
0              79               78             66              65   
1              63               58             63              60   
2              61               58             65              64   
3              75               72             71              66   
4              61               63             60              58   

   Side_foodWidth  Side_foodHeigth  Top_foodWidth  Top_foodHeigth  \
0             232              187            202             222   
1             192              181            204             217   
2             175              181            204             224   
3             201              206            222             208   
4             186              166            197             189   

   Side_coinForeground_pixel  Top_coinForeground_pixel    ...     tomato  \
0                       4411                      2915    ...          0   
1                       2474                      2840    ...          0   
2                       2575                      2695    ...          0   
3                       4004                      2900    ...          0   
4                       2734                      2150    ...          0   

   top_ratio  side_ratio  top_ratio_pixels  side_ratio_pixels  top_coin_ratio  \
0  10.453147    7.040571         11.860034           8.148266        0.019084   
1  11.711111    9.510673         11.620775          11.613985        0.020325   
2  10.984615    8.952798         13.673469          10.281553        0.019380   
3   9.854033    7.667778         12.918276           8.046703        0.018248   
4  10.699138    8.034348         12.748837           9.353328        0.021186   

   side_coin_ratio      top_area    side_area      kcal  
0         0.015924  10149.835465  5629.050304  0.650636  
1         0.020661  10455.862848  8339.033519  0.650636  
2         0.021008  12108.970100  6841.768989  0.650636  
3         0.017007  10885.297559  5666.357087  0.650636  
4         0.020161  10056.725759  5822.446964  0.650636  

[5 rows x 46 columns]

In [33]:
# Drop useless features
a = ['Top_coinWidth', 'Top_coinHeigth', 'Side_coinWidth', 'Side_coinHeigth', 'Top_foodHeigth', 'Side_foodHeigth', 'Top_foodWidth', 'Side_foodWidth', 'Side_coinForeground_pixel', 'Top_coinForeground_pixel', 'Side_foodForeground_pixel', 'Top_foodForeground_pixel']
useless_features = ['realVolume',  'real_calorie']
y = dataset['realVolume']
#y = dataset['real_calorie']
temp_X = dataset
X = dataset.drop(useless_features, axis=1)

In [34]:
# Split the dataset to trainand test data
train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.2, random_state=180)

In [35]:
train_X.columns

Index(['Side_coinWidth', 'Side_coinHeigth', 'Top_coinWidth', 'Top_coinHeigth',
       'Side_foodWidth', 'Side_foodHeigth', 'Top_foodWidth', 'Top_foodHeigth',
       'Side_coinForeground_pixel', 'Top_coinForeground_pixel',
       'Side_foodForeground_pixel', 'Top_foodForeground_pixel', 'column',
       'ellipsoid', 'irregular', 'realDensity', 'apple', 'banana', 'bread',
       'bun', 'doughnut', 'egg', 'fired_dough_twist', 'grape', 'lemon',
       'litchi', 'mango', 'mooncake', 'orange', 'peach', 'pear', 'plum',
       'qiwi', 'sachima', 'tomato', 'top_ratio', 'side_ratio',
       'top_ratio_pixels', 'side_ratio_pixels', 'top_coin_ratio',
       'side_coin_ratio', 'top_area', 'side_area', 'kcal'],
      dtype='object')

In [36]:
# Hyperparameters for random forest
parameters = {'bootstrap': [True],
              'random_state' : [42],
              'min_samples_leaf': [3, 7, 15],
              'criterion' : ['mse'],
              'n_estimators': [50, 100, 300], 
              'min_samples_split': [2, 5, 10, 20],
              'max_features': ['auto'],
              'max_depth': [5, 50, 100, None],
              'max_leaf_nodes': [10, 100, None]}

In [37]:
# Hyperparameters for random forest
params = {'bootstrap': True,
              'random_state' : 42,
              'min_samples_leaf': 2,
              'criterion' : 'mse',
              'n_estimators': 400 ,
              'min_samples_split': 2,
              'max_features': 'auto',
              'max_depth': None,
              'max_leaf_nodes': None}

In [38]:
# Create and train the model
rf_model = RandomForestRegressor(**params)
rf_model.fit(train_X, train_y)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=2, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=400, n_jobs=None,
           oob_score=False, random_state=42, verbose=0, warm_start=False)

In [16]:
# Grid search
grid_model = GridSearchCV(estimator=rf_model, param_grid=parameters, cv= 5)
grid_model.fit(train_X, train_y)

KeyboardInterrupt: 

In [40]:
from collections import defaultdict



# Get the accuracy of the model
rf_predictions = rf_model.predict(test_X)
score = mean_squared_error(test_y ,rf_predictions)
print(score)
print(math.sqrt(score))

actual = defaultdict(int)
predicted = defaultdict(int)

predicted_calories = defaultdict(int)
actual_calories = defaultdict(int)


i = 0
for pair in test_y.items():
    actual.append(pair[1])
    
    predicted_calories.append(dataset.loc[pair[0]]['realDensity'] * rf_predictions[i] * dataset.loc[pair[0]]['kcal'])
    actual_calories.append(dataset.loc[pair[0]]['real_calorie'])
    
    predicted.append(rf_predictions[i])
    print("actual : " + str(pair[1]) + " predicted : " + str(rf_predictions[i]))
    i += 1


135.34483405674027
11.63377986970444
actual : 90 predicted : 104.74073412698414
actual : 60 predicted : 56.79916666666667
actual : 150 predicted : 150.46166666666667
actual : 330 predicted : 329.9683333333333
actual : 90 predicted : 91.85041666666667
actual : 100 predicted : 113.51817460317461
actual : 90 predicted : 97.57083333333333
actual : 100 predicted : 100.25625
actual : 160 predicted : 159.47863095238094
actual : 100 predicted : 99.86125
actual : 170 predicted : 166.4128273809523
actual : 200 predicted : 212.00755681818183
actual : 180 predicted : 179.06892857142856
actual : 140 predicted : 145.13761904761904
actual : 160 predicted : 160.88791666666665
actual : 270 predicted : 262.4220833333334
actual : 190 predicted : 177.52461309523812
actual : 50 predicted : 49.13833333333333
actual : 220 predicted : 224.7971726190476
actual : 260 predicted : 257.71875000000006
actual : 170 predicted : 183.58979166666668
actual : 330 predicted : 327.25666666666666
actual : 150 predicted : 15

In [41]:
mse = mean_squared_error(actual_calories, predicted_calories)
print(mse)
print(math.sqrt(mse))

1215.7241597805923
34.86723619360434


In [112]:
test_X

Side_coinWidth  Side_coinHeigth  Top_coinWidth  Top_coinHeigth  \
394               64               63             68              66   
1345              64               61             67              65   
363               82               79             68              67   
604              114              115             67              68   
128               71               68             63              60   
1019              68               66             61              64   
833               74               72             74              71   
1268             118              110             67              64   
1298              67               61             66              67   
1221              66               64             79              75   
647               66               65             56              58   
694               73               69             72              71   
658               78               77             55              55   
1305              89               87             82              82   
882              123              119             68              72   
113               66               63             59              60   
959               81               77             71              67   
108               66               65             60              59   
475               83               80             67              67   
1333              66               66             68              67   
534               64               62             74              70   
935               76               74             59              58   
1274              73               71             61              60   
330               75               73             68              66   
765               72               69             60              58   
1372              68               63             75              72   
649               66               66             60              61   
1252              73               73             68              66   
100              122              111             90              90   
884              103               99             74              70   
...              ...              ...            ...             ...   
253               65               61             64              61   
704               63               62             71              70   
275               67               62             77              75   
35                62               57             69              67   
859               66               65             70              69   
1183              67               62             58              58   
811               72               69             65              62   
337               79               76             59              64   
932               75               70             60              60   
986               80               74             65              64   
211               66               66             72              67   
362               82               82             80              77   
960               83               77             71              69   
945               81               77             65              65   
1225              61               62             75              68   
307               63               60             69              69   
996               73               71             57              57   
538               66               63             61              60   
885               70               70             67              66   
519               61               64             68              68   
1121              70               66             62              63   
81                49               48             28              32   
313               62               59             59              57   
401               66               62         

In [49]:
# Visualize one of the trees from the random forest
estimator = rf_model.estimators_[8]

from sklearn.tree import export_graphviz
import os
# Export as dot file
export_graphviz(estimator, out_file='tree.dot', 
                feature_names = list(train_X),
                class_names = ['realVolume'],
                rounded = True, proportion = False, 
                precision = 2, filled = True)
os.system('dot -Tpng tree.dot -o tree.png')


"from subprocess import call\ncall(['dot', '-Tpng', 'tree.dot', '-o', 'tree.png', '-Gdpi=600'])\n\n# Display in jupyter notebook\nfrom IPython.display import Image\nImage(filename = 'tree.png')"